In [8]:
from ultralytics import YOLO
import numpy as np
import cv2
import pickle
import catcher_ex as CT
import torch
import SaveSystem_by_grip

print(torch.cuda.is_available())
Save_2_grip = SaveSystem_by_grip.save_system()

surface_model = YOLO("./cube_surface_seg2.pt")
cube_model = YOLO("./yolov8n-seg-custom.pt")
model_color_grip = YOLO("./grip_cube_color.pt")

with open('./hand_matrix/calibration.pkl', 'rb') as file:
    camera_matrix, dist_coeff = pickle.load(file)
    
CT = CT.block_detect(surface_model, cube_model, model_color_grip)

True


In [ ]:
#抓圖片
img = cv2.imread("./images/img0.png")

object_points = np.array([
    [0,0,0],#1
    [0,25,0],#2
    [25,25,0],#3
    [25,0,0],#4
    [0,0,25],#5
    [0,25,25],#6
    [25,25,25],#7
    [25,0,25],#8
], dtype=np.float32)

text_offset = 34
vertical_offset = 0
# text_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
for image_points in CT.detect_parts(img):
    # image_points = np.array(image_points, dtype=np.float32)
    for color_name, rgb in CT.get_color_text(img):
        image_points = np.float32(image_points)
        retval, rvec, tvec = cv2.solvePnP(object_points[:4], image_points, camera_matrix, dist_coeff)
        x = tvec[0]
        y = tvec[1]
        z = tvec[2]
        rotation_matrix, _ = cv2.Rodrigues(rvec)
        # 使用旋转矩阵计算欧拉角（roll-pitch-yaw 顺序）
        yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
        pitch = np.arctan2(
            -rotation_matrix[2, 0],
            np.sqrt(rotation_matrix[2, 1] ** 2 + rotation_matrix[2, 2] ** 2),
        )
        roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
        # 将弧度转换为度数
        rz = np.degrees(yaw)
        ry = np.degrees(pitch)
        rx = np.degrees(roll)
        RotationZ = np.array([rz])
        text_loc_tvec = (5, 15 + vertical_offset)
        text_loc_rvec = (5, 32 + vertical_offset)
        text_loc_check = (400, 15 + vertical_offset)
        # print(color_name)
        # print(f"{x=}")
        # print(f"{y=}")
        # print(f"{rx=}")
        xyz_str = [f"{c}: {v[0]:.2f}" for c, v in zip("xyz", [x, y, z])]
        cv2.putText(img, f"{color_name} {', '.join(xyz_str)}", text_loc_tvec, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
        cv2.putText(img, f"Rotate Z: {rz:.1f},   Rotate Y: {ry:.1f},   Rotate X: {rx:.1f}", text_loc_rvec, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
        if -10 < x < 10 and -10 < y < 10:
            cv2.putText(img, "OK", text_loc_check, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        else:
            cv2.putText(img, "Moving", text_loc_check, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255) ,2)
        cv2.circle(img, (int(x[0]), int(y[0])), 5, (0, 0, 255), -1)
        vertical_offset += 34

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
lens = cv2.VideoCapture(0,cv2.CAP_DSHOW)

In [11]:
lens.release()

In [10]:
object_points = np.array(
    [   [-25, -25, 0],  # 1
        [-25, 25, 0],  # 2
        [25, 25, 0],  # 3
        [25, -25, 0],  # 4
        
    ], dtype=np.float32)

Save_2_grip.reset()

offset_x = 0
offset_y = 0

while True:
    ret, img = lens.read()
    if not ret:
        break
    vertical_offset = 0
    for image_points in CT.detect_parts(img):
        for color_name, rgb in CT.get_color_text(img):
            image_points = np.float32(image_points)
            retval, rvec, tvec = cv2.solvePnP(object_points[:4], image_points, camera_matrix, dist_coeff)
            if not retval:
                break
            x = tvec[0] - offset_x
            y = tvec[1] - offset_y
            z = tvec[2]
            rotation_matrix, _ = cv2.Rodrigues(rvec)
            # 使用旋转矩阵计算欧拉角（roll-pitch-yaw 顺序）
            yaw = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
            pitch = np.arctan2(-rotation_matrix[2, 0],np.sqrt(rotation_matrix[2, 1] ** 2 + rotation_matrix[2, 2] ** 2))
            roll = np.arctan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
            # 将弧度转换为度数
            rz = np.degrees(yaw)
            ry = np.degrees(pitch)
            rx = np.degrees(roll)
            RotationZ = np.array([rz])
            text_loc_tvec = (5, 15 + vertical_offset)
            text_loc_rvec = (5, 32 + vertical_offset)
            text_loc_check = (400, 15 + vertical_offset)

            Save_2_grip.save_coordinate_by_color(color_name, x, y,RotationZ, 10)
            xyz_str = [f"{c}: {v[0]:.2f}" for c, v in zip("xyz", [x, y, z])]
            cv2.putText(img, f"{color_name} {', '.join(xyz_str)}", text_loc_tvec, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
            cv2.putText(img, f"Rotate Z: {rz:.1f},   Rotate Y: {ry:.1f},   Rotate X: {rx:.1f}", text_loc_rvec, cv2.FONT_HERSHEY_SIMPLEX, 0.5, rgb, 2)
            if -10 < x < 10 and -10 < y < 10:
                cv2.putText(img, "OK", text_loc_check, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                cv2.putText(img, "Moving", text_loc_check, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255) ,2)
            vertical_offset += 34

    else:
        pass
    cv2.imshow("test", img)
    key = cv2.waitKey(1)
    if key == 27:
        break
cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO('./cube_color.pt')
cap = cv2.VideoCapture(1,cv2.CAP_DSHOW)
while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame,conf=0.8)
        color = None
        if results == None:
            continue
        for result in results:
            for box in result.boxes:
                if box[0] == 0:
                    color = "red"
                elif box[0] == 1:
                    color = "black"
                elif box[0] == 2:
                    color = "yellow"
                elif box[0] == 3:
                    color = "green"
                elif box[0] == 4:
                    color = "purple"
                print(box[0])
                print(color)
                annotated_frame = results[0].plot(labels=True, verbose = False)


        cv2.imshow("YOLOv8 Inference", annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break


cap.release()
cv2.destroyAllWindows()